# Enhancement

## 1. Correlation analysis

In the following we will create a correlation matrix to analyze if there exists a correaltion between the attributes of interest, including case fatality rate (CFR), infection rate over population (IR), population density (PD), and Tendency to wear mask (TWM). For this analysis, we will focus on the latest state-level data.

Upsert the MongoDB database

In [ ]:
!python3 data_acquire.py

2020-12-06 15:36:06,026 [upsert_db]: us:rows=320, update=320, insert=0
2020-12-06 15:36:53,914 [upsert_db]: state:rows=15304, update=15304, insert=0
2020-12-06 15:36:56,786 [upsert_db]: county:rows=3142, update=3142, insert=0
2020-12-06 15:36:56,888 [upsert_db]: population:rows=55, update=55, insert=0
2020-12-06 15:36:57,438 [_worker]: main loop worker ignores exception and continues: HTTPSConnectionPool(host='raw.githubusercontent.com', port=443): Read timed out. (read timeout=0.5)


In [ ]:
import pandas as pd
import numpy as np
from database import fetch_all_db_as_df

df_dict = fetch_all_db_as_df()

### 1.1 Extract the latest state-level data 

In [ ]:
## Extract the latest Data
df = df_dict['covid-us-state']
df.fips = df.fips.apply(lambda x: str(x).zfill(2))
df = df[df.date == max(df.date)]
df = df.drop(columns='date', axis=1).reset_index(drop=True)
df.head()

###  1.2 Read population and area data, Merge with Covid data

In [13]:
state_pop = df_dict['state-population']
state_pop.head()

,state,total
0,Alabama,4903185
1,Alaska,731545
2,Arizona,7278717
3,Arkansas,3017804
4,California,39512223


In [14]:
## Area in sq. mile
state_area =  df_dict['state-area']
state_area.head()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming', 'Guam', 'Virgin Islands',
       'Northern Mariana Islands', 'Puerto Rico'], dtype=object)

In [ ]:
df_merged = 